# Sample Skyspark Batched

This is a modified version of the "Skyspark Validation" notebook. It allows a user to query for a set of similar equipment and validate each equipemnt in turn.

# 1) Setup

## Imports

In [ ]:
# ----------------------------------------
# Imports
# ----------------------------------------
import os
import json
import re

from rdflib import Namespace, SH, RDF, BNode, Graph
from pyshacl import validate
from dotenv import load_dotenv
load_dotenv()

from tasty import constants as tc
from tasty import graphs as tg
from tasty.skyspark import client as cl
from tasty.skyspark import process_graphs as pg
from tasty.skyspark import helpers

## Inputs
Define the key variables and input information here

***Items to Change***
- `equip_tag`: this is the equipment tag to use in the axon query to find the relavent equipment; this can also be an axon query in itself (e.g. 'vav and hotWateHeat')
- `building`: this is the building name to use in the axon query (should match the building's "dis" name in Skyspark)
- `SHAPE`: this is the name of the SHACL equipment shape against which you would like to validate your sample equipment in the instance data
- `input_namespace_uri`: this is the namespace uri used for your sample equipment in the instance data
- `raw_data_graph_filename`: this is the filename/filepath to save the raw instance data (in turtle format) retrieved from the Skyspark API call
- `data_graph_filename`: this is the filename/filepath to save the cleaned/processed instance data for the data graph to be used for validation
- `shapes_graph_filename`: this it the filename/filepath of the SHACL shapes data for the shape graph 
***Remaining Items*** </br>
These items should be okay as is, but can be changed if need be. If you are printing out results, <u>*make sure that the output directory exists in your local file structure*</u>.
- `output_directory`: this is the directory where output files will be printed to below
- `tasty_main_directory`: this is the absolute path of the main tasty directory. It should just be the parent directory of the current working directory.

In [ ]:
# ----------------------------------------
# User Defined Variables
# ----------------------------------------

equip_tag = 'fcu'
# equip_tag = 'ahu'
building = 'OTF'

SHAPE = 'NREL-FCU-CS-HW-CHW-Shape'
# SHAPE = 'NREL-AHU-VAV-MZ-HW-CHW-Evap-Shape'
input_namespace_uri = 'urn:/_#'

raw_data_graph_filename = 'examples/output/sample_skyspark_vav_raw.ttl'
data_graph_filename = 'examples/output/sample_skyspark_vav_clean.ttl'
shapes_graph_filename = 'tasty/generated_shapes/haystack_all.ttl'
validation_results_output_filename = 'examples/output/OTF_FCU_results.txt'

output_directory = os.path.join(os.path.abspath(''), 'example_data/output')
tasty_main_directory = os.path.join(os.path.abspath(''), '../')
# print(tasty_main_directory)

# ----------------------------------------
# Variables and Constants
# ----------------------------------------
skyspark_api_url = os.environ.get('API_ENDPOINT')

NAMESPACE = Namespace(input_namespace_uri)
shape_name = tc.PH_SHAPES_NREL[SHAPE]
output_file = os.path.join(tasty_main_directory, validation_results_output_filename)

## API Request From Skyspark 
NOTE - Must be connected to NREL network to access the api endpoint

In [ ]:
client = cl.SkysparkClient(skyspark_api_url)

make query for list of equipment with equip_tag (defined above)

In [ ]:
from IPython.display import JSON

axon_query_string = client.generate_axon_query_for_equip_type(equip_tag, building)
print(axon_query_string)

response = client.make_get_request(axon_query_string, 'json')

print(response.status_code, end = " - ")
if response.status_code == 200:
    print("Sucess")
elif response.status_code == 404:
    print("Not Found")

raw_skyspark_data = json.loads(response.text)
JSON(raw_skyspark_data['rows'], expanded=True)

In [ ]:
equip_list = []

for item in raw_skyspark_data['rows']:
    new_equip = item['navName']
    equip_list.append(new_equip)
for equip in equip_list:
    print(equip)

### Define the Validate Equip Function

In [ ]:
def validate_equip(equip_name, building):
    helpers.append_data_to_file(f"Equipment Name:\t{equip_name} \n", output_file) 
    # ----------------------------------------
    # Get Axon Query
    # ----------------------------------------
    axon_query_string = client.generate_axon_query_for_equip(equip_name, building)
    print("Generated axon query: " + axon_query_string)

    # ----------------------------------------
    # Perform Request
    # ----------------------------------------
    response = client.make_get_request(axon_query_string, 'turtle')

    print(response.status_code, end = " - ")
    if response.status_code == 200:
        print("Sucess")
    elif response.status_code == 404:
        print("Not Found")

    raw_skyspark_data = response.text
    # print(raw_skyspark_data)

    # ----------------------------------------
    # save Response to File
    # ----------------------------------------
    f = os.path.join(tasty_main_directory, raw_data_graph_filename)
    helpers.save_data_to_file(raw_skyspark_data, f)
    print(f"raw instance data saved to '{raw_data_graph_filename}' ")

    # ----------------------------------------
    # Get Equip ID
    # ----------------------------------------
    equip_id = client.get_equip_id(equip_name, building)
    print("Equipment id: " + equip_id)
    helpers.append_data_to_file(f"Equipment ID:\t{equip_id}\n", output_file) 
    target_node = NAMESPACE[equip_id]
    # ----------------------------------------
    # Create instance of SkysparkGraphProcessor
    # ----------------------------------------
    schema = tc.HAYSTACK
    version = tc.V3_9_10
    sgp = pg.SkysparkGraphProcessor(input_namespace_uri,schema, version)

    # ----------------------------------------
    # Pre Process raw skyspark .ttl file 
    # ----------------------------------------
    f1 = os.path.join(tasty_main_directory, raw_data_graph_filename)
    f2 = os.path.join(tasty_main_directory, data_graph_filename)
    sgp.clean_raw_skyspark_turtle(f1,f2)
    print(f"cleaned instance data saved to '{data_graph_filename}' ")

    # ----------------------------------------
    # Generate Graphs
    # ----------------------------------------

    # Data Graph
    dg_file = os.path.join(tasty_main_directory, data_graph_filename)
    data_graph = sgp.get_data_graph(dg_file)
    print("...loaded data graph")

    # Shapes Graph
    sg_file = os.path.join(tasty_main_directory, shapes_graph_filename)
    shapes_graph = sgp.get_shapes_graph(sg_file, target_node, shape_name)
    print("...loaded shapes graph")
    helpers.append_data_to_file(f"SHACL Shape:\t{shape_name}\n", output_file) 

    # Ontology Graph
    ont_graph = sgp.get_ontology_graph()
    print("...loaded ontology graph")

    # ----------------------------------------
    # Run pySCHACL Validation
    # ----------------------------------------    
    result = validate(data_graph, shacl_graph=shapes_graph, ont_graph=ont_graph)
    conforms, results_graph, results = result
    
    print(f"Conforms: {conforms}")
    helpers.append_data_to_file(f"Conforms:\t\t{conforms}\n", output_file) 

    # ----------------------------------------
    # Determine Missing Points
    # ----------------------------------------
    missing_points = sgp.determine_missing_points(results_graph)
    
    report_string = ""
    if len(missing_points['required']) <= 0:
        print("No Required Points Missing")
        report_string += "No Required Points Missing\n"
    else:
        print(f"{len(missing_points['required'])} Missing Required Points:")
        report_string += f"{len(missing_points['required'])} Missing Required Points:\n"
        for point in missing_points['required']:
            print(f"\t{point}")
            point_trunc = point[point.rfind('#') + 1:]
            report_string += f"\t{point_trunc}\n"

    if len(missing_points['optional']) <= 0:
        print("No Optional Points Missing")
        report_string += "No Optional Points Missing\n"
    else:
        print(f"{len(missing_points['optional'])} Missing Optional Points:")
        report_string += f"{len(missing_points['optional'])} Missing Optional Points:\n"
        for point in missing_points['optional']:
            print(f"\t{point}")
            point_trunc = point[point.rfind('#') + 1:]
            report_string += f"\t{point_trunc}\n"
    helpers.append_data_to_file(report_string, output_file) 

In [ ]:
helpers.save_data_to_file("", output_file) # clear the output file
for equip in equip_list:
    validate_equip(equip, building)